This is to explore the eodms api for my project

In [21]:
from eodms_rapi import EODMSRAPI
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, MultiPolygon
import json

file = open('C:/Users/User/Desktop/nasashit/credentials.txt') # my credentials (not rly important)
temp = file.read().splitlines()
rapi = EODMSRAPI(temp[0], temp[1])

bloom_df = gpd.read_file("WildflowerBlooms_AreaOfInterest.geojson")
print(bloom_df.head())
print(bloom_df['geometry'][3]) # it is really deatiled which will cause problems for the API

     id                                      Site  Type  Season        Area  \
0  None                               Chino Hills  Wild  Spring   25382.057   
1  None           Carrizo Plain National Monument  Wild  Spring  354751.514   
2  None  Antelope Valley California Poppy Reserve  Wild  Spring   25182.333   
3  None         Laura and Jack Dangermond Reserve  Wild  Spring    9027.693   
4  None                          Sedgwick Reserve  Wild  Spring    9511.600   

                                            geometry  
0  MULTIPOLYGON (((-117.68286 33.87223, -117.7196...  
1  MULTIPOLYGON (((-119.50857 34.87044, -119.6043...  
2  MULTIPOLYGON (((-118.50267 34.69779, -118.5023...  
3  MULTIPOLYGON (((-120.46129 34.56679, -120.4353...  
4  MULTIPOLYGON (((-119.95777 34.73243, -119.9831...  
MULTIPOLYGON (((-120.46129269331323 34.56678791539294, -120.43533672100645 34.56297086064195, -120.43533672100645 34.56297086064195, -120.3857150092435 34.53434295000947, -120.3899137694696 34.52

After severe testing and research and iterations, I have found that images and data from the collections Radarsat2RawProducts provides data that correlate with my ground truth data which are the ones close to california. The other problem was the geometry of the ground truth data as it is very complex so used simplify method to test things out, below is the edited code after a lot of trials and errors.


In [49]:
selected_site = bloom_df.iloc[1] # testing for Carrizo Plain National Monument
site_geometry = selected_site.geometry

# Test different simplification levels for your bloom site geometry
simplification_levels = [0.01, 0.05, 0.1, 0.2, 0.5]

for tolerance in simplification_levels:
    print(f"\n🔧 Testing bloom site geometry with tolerance: {tolerance}")
    
    # Simplify the geometry
    simplified_geometry = site_geometry.simplify(tolerance=tolerance, preserve_topology=True)
    
    def geometry_to_geojson_dict(geom):
        """Converts a Shapely geometry to a GeoJSON dictionary using the official Shapely function."""
        geojson_string = shapely.to_geojson(geom)
        geojson_dict = json.loads(geojson_string)
        return geojson_dict

    geometry_geojson = geometry_to_geojson_dict(simplified_geometry)
    
    feat = [('INTERSECTS', geometry_geojson)]
    
    dates = [{"start": "20230301_000000", "end": "20230601_000000"}]  # Spring 2023 bloom season
    filters = {}
    
    # Test with Radarsat2RawProducts (the collection that worked best)
    rapi.search("Radarsat2RawProducts", filters=filters, features=feat, dates=dates, max_results=100)
    results = rapi.get_results('full')
    print(f"Tolerance {tolerance}: Found {len(results)} scenes with Radarsat2RawProducts")
    
    if len(results) > 0:
        print("✅ SUCCESS! Bloom site geometry works with this tolerance")
        break

# using cali as the fallback
if len(results) == 0:
    print("\n🔄 Using California bounding box as fallback...")
    feat = [('INTERSECTS', {
        "type": "Polygon",
        "coordinates": [[
            [-124.4, 32.5],
            [-114.1, 32.5],  #this is californias coordinates
            [-114.1, 42.0],
            [-124.4, 42.0],
            [-124.4, 32.5]
        ]]
    })]
    
    collections = ['Radarsat2RawProducts']  # Just test the best collection
    for collection in collections: 
        rapi.search(collection, filters=filters, features=feat, dates=dates, max_results=1000)
        results = rapi.get_results('full')
        print(f"California bbox with {collection}: Found {len(results)} scenes")


🔧 Testing bloom site geometry with tolerance: 0.01

| EODMSRAPI | 2025-10-01 11:45:39 | Searching for Radarsat2RawProducts images on RAPI
| EODMSRAPI | 2025-10-01 11:45:39 | Querying records...
| EODMSRAPI | 2025-10-01 11:45:39 | RAPI Query URL: https://www.eodms-sgdot.nrcan-rncan.gc.ca/wes/rapi/search?collection=Radarsat2RawProducts&query=%28CATALOG_IMAGE.START_DATETIME%3E%3D%272023-03-01T00%3A00%3A00Z%27+AND+CATALOG_IMAGE.START_DATETIME%3C%3D%272023-06-01T00%3A00%3A00Z%27%29+AND+%28CATALOG_IMAGE.THE_GEOM_4326+INTERSECTS+POLYGON+%28%28-119.50857432459836+34.87043932367128%2C+-119.60433286498474+34.80485128231074%2C+-119.91128489855204+34.91897447427807%2C+-119.96900237494931+34.96226258157603%2C+-120.01491400390168+35.00292716721956%2C+-120.0647609153357+35.09606218595152%2C+-120.33171998482592+35.286150846049985%2C+-120.24423085253207+35.475071589596666%2C+-119.96794412160605+35.481196036658204%2C+-119.53028720564085+35.155843536865184%2C+-119.3866036143617+35.03032683643743%2C+-119

| EODMSRAPI | Fetching result metadata:   0%|          | 0/1908 [00:00<?, ?item/s]

Tolerance 0.01: Found 1908 scenes with Radarsat2RawProducts
✅ SUCCESS! Bloom site geometry works with this tolerance


In [53]:
df = pd.json_normalize(results)

# Save to CSV
csv_filename = "bloomwatch_scenes_metadata.csv"
df.to_csv(csv_filename, index=False)

print(f"✅ Saved {len(df)} scenes to '{csv_filename}'")
print(f"📊 CSV has {len(df.columns)} columns")

# Show the column names to see what data you have
print("\n📋 Available columns:")
for col in df.columns.tolist()[:20]:  # First 20 columns
    print(f"  - {col}")

print(df)

✅ Saved 1908 scenes to 'bloomwatch_scenes_metadata.csv'
📊 CSV has 119 columns

📋 Available columns:
  - recordId
  - collectionId
  - clientOrderNumber
  - vapEnabled
  - withinOrbitTube
  - processingFacility
  - clientOrderItemNumber
  - beamModeVersion
  - beamMnemonic
  - providerId
  - quicklookFullName
  - productDatum
  - orbitDataSource
  - processingLevel
  - incidenceAngle
  - eulaFilename
  - numberOfAzimuthLooks
  - productType
  - polarization
  - beamModeType
      recordId          collectionId clientOrderNumber vapEnabled  \
0     16678409      RCMImageProducts                         true   
1     16678427      RCMImageProducts                         true   
2     16673398      RCMImageProducts                         true   
3     16639864      RCMImageProducts                         true   
4     16639742      RCMImageProducts                         true   
...        ...                   ...               ...        ...   
1903  15933895  Radarsat2RawProducts   

In [59]:
import numpy as np
from datetime import datetime

# Load your CSV
df = pd.read_csv("bloomwatch_scenes_metadata.csv")

print("🎯 BLOOMWATCH SCENE ANALYSIS")
print("=" * 50)

# Fix date columns - handle any format issues
def safe_date_convert(date_str):
    try:
        # Extract just the date part (before 'T' or space)
        if pd.isna(date_str):
            return pd.NaT
        date_part = str(date_str).split('T')[0].split(' ')[0]
        return pd.to_datetime(date_part)
    except:
        return pd.NaT

df['acquisition_date'] = df['acquisitionStartDate'].apply(safe_date_convert)
df['acquisition_date'] = pd.to_datetime(df['acquisition_date'])

print(f"📊 Total scenes: {len(df):,}")
print(f"📅 Date range: {df['acquisition_date'].min().strftime('%Y-%m-%d')} to {df['acquisition_date'].max().strftime('%Y-%m-%d')}")
print(f"🛰️ Collections: {df['collectionId'].unique().tolist()}")

# Polarization analysis
print(f"\n📡 Polarizations available:")
polarization_counts = df['polarization'].value_counts()
for pol, count in polarization_counts.items():
    print(f"   {pol}: {count} scenes")

# Resolution analysis
print(f"\n🔍 Spatial resolutions:")
resolution_counts = df['spatialResolution'].value_counts().head(10)
for res, count in resolution_counts.items():
    print(f"   {res}: {count} scenes")





🎯 BLOOMWATCH SCENE ANALYSIS
📊 Total scenes: 1,908
📅 Date range: 2023-03-02 to 2023-05-30
🛰️ Collections: ['RCMImageProducts', 'Radarsat2RawProducts']

📡 Polarizations available:
   HH: 1720 scenes
   HH HV: 88 scenes
   VV VH: 47 scenes
   HH VV HV VH: 28 scenes
   CH CV: 17 scenes
   VV: 8 scenes

🔍 Spatial resolutions:
   2: 680 scenes
   6: 332 scenes
   40: 314 scenes
   3: 278 scenes
   8: 136 scenes
   50: 73 scenes
   30: 58 scenes
   1: 25 scenes
   100: 12 scenes


In [60]:
# analysing the temps
# Analyze scenes by month for bloom season
print(f"\n🌸 BLOOM SEASON COVERAGE (March-May 2023):")
bloom_season = df[
    (df['acquisition_date'].dt.year == 2023) & 
    (df['acquisition_date'].dt.month.isin([3, 4, 5]))
]

if len(bloom_season) > 0:
    monthly_counts = bloom_season['acquisition_date'].dt.month.value_counts().sort_index()
    month_names = {3: 'March', 4: 'April', 5: 'May'}
    for month, count in monthly_counts.items():
        print(f"   {month_names.get(month, month)}: {count} scenes")
else:
    print("   No scenes found in bloom season 2023")
    
# Check other years too
print(f"\n📈 YEARLY COVERAGE:")
yearly_counts = df['acquisition_date'].dt.year.value_counts().sort_index()
for year, count in yearly_counts.items():
    print(f"   {year}: {count} scenes")


🌸 BLOOM SEASON COVERAGE (March-May 2023):
   March: 17 scenes
   April: 43 scenes
   May: 42 scenes

📈 YEARLY COVERAGE:
   2023.0: 102 scenes


In [61]:
# Find the best scenes for bloom detection
print(f"\n🌟 FINDING OPTIMAL SCENES FOR ML:")

# Convert spatialResolution to numeric safely
df['resolution_num'] = pd.to_numeric(df['spatialResolution'], errors='coerce')

optimal_criteria = df[
    (df['polarization'].str.contains('HV|VH', na=False)) &  # Vegetation-sensitive
    (df['resolution_num'] <= 100) &  # Reasonable resolution
    (df['acquisition_date'].dt.year >= 2020)  # Recent data
].copy()

print(f"   Scenes with HV/VH polarization: {len(optimal_criteria)}")

if len(optimal_criteria) > 0:
    print(f"   Best polarization distribution:")
    best_pol_counts = optimal_criteria['polarization'].value_counts()
    for pol, count in best_pol_counts.items():
        print(f"     {pol}: {count} scenes")
        
    # Show date distribution of optimal scenes
    print(f"\n   📅 Optimal scenes by year:")
    optimal_years = optimal_criteria['acquisition_date'].dt.year.value_counts().sort_index()
    for year, count in optimal_years.items():
        print(f"     {year}: {count} scenes")


🌟 FINDING OPTIMAL SCENES FOR ML:
   Scenes with HV/VH polarization: 85
   Best polarization distribution:
     HH HV: 85 scenes

   📅 Optimal scenes by year:
     2023: 85 scenes


Now that sample data collection has ended, lets extract all the useful data depending on the ground truth geometries

ok seems like I cant really work with this data there is no 